# 제로베이스데이터사이언스 스쿨 원데이 프로젝트
> ## 세계 테러 데이터 분석 - SQL 사용하기

---

#### 총 8문제 100점 만점 

- 문제 1 10점  
- 문제 2,3,5,6,7,8번 각 10점 
- 문제 4번 30점
- 총점 100점 

#### 유의사항

- 모든 정답 코드는 jupyter notebook 파일에 작성해주세요. 
- 1번 문제는 정답 코드를 그대로 공개해드립니다. 따라서 진행해주세요. 
- 그 외 문제들은 참고하실 수 있도록 주석과 출력 코드를 보여드립니다. 
- 제출시, 반드시 결과가 실행되어 있어야 합니다. 미실행 또는 오류시 감점입니다. 
- 제출시, [DS]sql3_choyongha.ipynb 형식에 맞춰 제출해주세요.


---

여러분은 이전 과제에서 세계 테러 데이터 분석 프로젝트를 진행하였습니다. 

이번 시간에는 분석할 데이터를 Database 로 변환하고, 분석에 필요한 데이터를 저장된 Database 에서 쿼리하여 가져와 보도록 하겠습니다. 

#### 작업환경

- aws rds 사용시: 이전 과제에서 생성한 AWS RDS (MySQL) 의 Database 및 사용자를 사용하세요. 
  - Database Name: oneday 
  - User Name / Password: oneday / 1234
- local 사용시: 로컬 환경에서 생성한 Database 및 사용자를 사용하세요.
  - 위와 동일하게 생성해주세요

## 원본 Data Download
- https://www.kaggle.com/datasets/START-UMD/gtd

---

문제 1.  csv 파일에 저장된 세계 테러 데이터를 하나의 테이블에 저장하세요. 

- globalterrorismmdb_0718.csv - https://www.kaggle.com/datasets/START-UMD/gtd
- pandas의 to_sql 함수 사용 (힌트 : create_engine)
- Table name : origin_terror_data 

제출 1. Table 생성 및 조회 결과 : DESC origin_terror_data, SELECT  COUNT(*) FROM origin_terror_data

제출 2. 관련 코드 및 실행 결과 (ipynb)



In [4]:
##############################################################################################
#  문제 1. 원본 데이터 (csv)를 데이터베이스로 변환하고 확인하기 
#  globalterrorismdb_0718dist.csv -> origin_terror_data (table)
#  Pandas 의 to_sql 사용 
##############################################################################################

##############################################################################################
#  pip install pymysql
#  pip install SQLAlchemy Flask-SQLAlchemy
##############################################################################################

import time
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import configparser

# 데이터 불러오기
df = pd.read_csv('./datas/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')

# DB 접속 엔진 객체 생성
user = 'oneday'
password = 'zerobase'
host = 'oneday.cg9o7spxrfwt.ap-northeast-2.rds.amazonaws.com'
port = 3306
database = 'oneday'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4')

# DB 테이블 명(생성될 테이블 이름)
table_name = "origin_terror_data"

# DB에 DataFrame 적재
df.to_sql(index = False, 
          name = table_name,
          con = engine,
          if_exists = 'append',
          method = 'multi', 
          chunksize = 10000)

C:\Users\User\AppData\Local\Temp\ipykernel_18840\2218414628.py:19: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./datas/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')


181691

In [5]:
desc = engine.execute("DESC origin_terror_data").fetchall()
records = engine.execute("SELECT COUNT(*) FROM origin_terror_data").fetchall()
print(records)

[(181691,)]


문제 2. origin_terror_data 에서 region, country 관련 데이터는 code 와 txt (name) 속성으로 정의되어 있습니다. 

문제 2-1. Region 및 Country 테이블을 그림과 같은 구조로 생성하세요. 

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 
- Region 과 Country 데이터 사이의 관계를 파악하여 Foreign Key 를 설정하세요.

참고> 
- Region.region_code     = origin_terror_data.region
- Region.region_name     = origin_terror_data.region_txt
- Country.country_code   = origin_terror_data.country
- Country.country_name   = origin_terror_data.country_txt


In [6]:
##############################################################################################
#  문제 2. region / country / city 데이터 추출하여 데이터베이스로 변환하기 
#  중복을 제거한 code - name 값 
#  region - country - city 관계 정의 
##############################################################################################
import mysql.connector

conn = mysql.connector.connect(
    host = "oneday.cg9o7spxrfwt.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "oneday",
    password = "zerobase",
    database = "oneday"
)
cursor = conn.cursor(buffered=True)

In [10]:
# region_txt 의 max length 체크 
cursor.execute("SELECT max(LENGTH(region_txt)) FROM origin_terror_data")
max_length = cursor.fetchall()
print(max_length)

[(27,)]


In [11]:
# Region 테이블 만들기
cursor.execute("""create table Region(
                region_code int PRIMARY KEY auto_increment,
                region_name varchar(32)
                )
             """)

In [12]:
# country_txt 의 max length 체크 
cursor.execute("SELECT max(LENGTH(country_txt)) FROM origin_terror_data")
max_length = cursor.fetchall()
print(max_length)

[(32,)]


In [13]:
# Country 테이블 만들기 
cursor.execute("""create table Country(
                country_code int PRIMARY KEY auto_increment,
                region_code int,
                FOREIGN KEY(region_code) REFERENCES Region(region_code),
                country_name varchar(32)
                )
             """)

문제 2-2. origin_terror_data 테이블에서 Region 및 Country 데이터를 추출하여 문제 2-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요. 
- 데이터를 INSERT 할때 순서를 고민하세요. 

제출 3. Table 생성 및 조회 결과 : DESC Region, DESC Country

제출 4. 관련 코드 및 실행 결과 (ipynb)

In [14]:
# Region 데이터 추출하기 
cursor.execute("SELECT distinct(region), region_txt FROM origin_terror_data ORDER BY region")
region_result = cursor.fetchall()
region_result = pd.DataFrame(region_result)
region_result

,0,1
0,1,North America
1,2,Central America & Caribbean
2,3,South America
3,4,East Asia
4,5,Southeast Asia
5,6,South Asia
6,7,Central Asia
7,8,Western Europe
8,9,Eastern Europe
9,10,Middle East & North Africa


In [15]:
# Region 테이블에 INSERT 
sql = "insert into Region(region_code, region_name) values (%s, %s)"
for i, row in region_result.iterrows():
    cursor.execute(sql, tuple(row))
    print(row)
conn.commit()

0                1
1    North America
Name: 0, dtype: object
0                              2
1    Central America & Caribbean
Name: 1, dtype: object
0                3
1    South America
Name: 2, dtype: object
0            4
1    East Asia
Name: 3, dtype: object
0                 5
1    Southeast Asia
Name: 4, dtype: object
0             6
1    South Asia
Name: 5, dtype: object
0               7
1    Central Asia
Name: 6, dtype: object
0                 8
1    Western Europe
Name: 7, dtype: object
0                 9
1    Eastern Europe
Name: 8, dtype: object
0                            10
1    Middle East & North Africa
Name: 9, dtype: object
0                    11
1    Sub-Saharan Africa
Name: 10, dtype: object
0                       12
1    Australasia & Oceania
Name: 11, dtype: object


In [16]:
# Country 데이터 추출하기 
cursor.execute("SELECT distinct(country), region, country_txt FROM origin_terror_data ORDER BY country")
country_result = cursor.fetchall()
country_result = pd.DataFrame(country_result)
country_result

,0,1,2
0,4,6,Afghanistan
1,5,9,Albania
2,6,10,Algeria
3,7,8,Andorra
4,8,11,Angola
...,...,...,...
200,605,11,People's Republic of the Congo
201,1001,9,Serbia
202,1002,9,Montenegro
203,1003,9,Kosovo


In [17]:
# Country 테이블에 INSERT 
sql = "insert into Country(country_code, region_code, country_name) values (%s, %s, %s)"
for i, row in country_result.iterrows():
    cursor.execute(sql, tuple(row))
    print(row)
conn.commit()

0              4
1              6
2    Afghanistan
Name: 0, dtype: object
0          5
1          9
2    Albania
Name: 1, dtype: object
0          6
1         10
2    Algeria
Name: 2, dtype: object
0          7
1          8
2    Andorra
Name: 3, dtype: object
0         8
1        11
2    Angola
Name: 4, dtype: object
0                     10
1                      2
2    Antigua and Barbuda
Name: 5, dtype: object
0           11
1            3
2    Argentina
Name: 6, dtype: object
0         12
1          7
2    Armenia
Name: 7, dtype: object
0           14
1           12
2    Australia
Name: 8, dtype: object
0         15
1          8
2    Austria
Name: 9, dtype: object
0            16
1             7
2    Azerbaijan
Name: 10, dtype: object
0         17
1          2
2    Bahamas
Name: 11, dtype: object
0         18
1         10
2    Bahrain
Name: 12, dtype: object
0            19
1             6
2    Bangladesh
Name: 13, dtype: object
0          20
1           2
2    Barbados
Name: 14, d

0      164
1       10
2    Qatar
Name: 135, dtype: object
0        166
1          9
2    Romania
Name: 136, dtype: object
0       167
1         9
2    Russia
Name: 137, dtype: object
0       168
1        11
2    Rwanda
Name: 138, dtype: object
0             173
1              10
2    Saudi Arabia
Name: 139, dtype: object
0        174
1         11
2    Senegal
Name: 140, dtype: object
0                  175
1                    9
2    Serbia-Montenegro
Name: 141, dtype: object
0           176
1            11
2    Seychelles
Name: 142, dtype: object
0             177
1              11
2    Sierra Leone
Name: 143, dtype: object
0          178
1            5
2    Singapore
Name: 144, dtype: object
0                179
1                  9
2    Slovak Republic
Name: 145, dtype: object
0         180
1           9
2    Slovenia
Name: 146, dtype: object
0                181
1                 12
2    Solomon Islands
Name: 147, dtype: object
0        182
1         11
2    Somalia
Name: 148, dtyp

문제 3.  origin_terror_data 에서 attack type, target type, weapon type 관련 데이터는 code 와 txt 속성으로 정의되어 있습니다.  

문제 3-1. AttackType, TargetType, WeaponType 테이블을 그림과 같은 구조로 생성하세요. 

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 

참고>
- AttackType.attacktype_code    = origin_terror_data.attacktype1
- AttackType.attacktype_desc    = origin_terror_data.attacktype1_txt 
- TargetType.targtype_code      = origin_terror_data.targtype1
- TargetType.targtype_desc      = origin_terror_data.targtype1_txt
- WeaponType.weaptype_code      = origin_terror_data.weaptype1
- WeaponType.weaptype_desc      = origin_terror_data.weaptype1_txt 

In [20]:
# attacktype1 의 max length 체크 
cursor.execute("SELECT max(LENGTH(attacktype1_txt)) FROM origin_terror_data")
max_length = cursor.fetchall()
print(max_length)

[(35,)]


In [33]:
# AttackType 테이블 만들기 
cursor.execute("""create table AttackType(
                attacktype_code int PRIMARY KEY auto_increment,
                attacktype_desc varchar(35)
                )
             """)

In [34]:
# targettype1 의 max length 체크 
cursor.execute("SELECT max(LENGTH(targtype1_txt)) FROM origin_terror_data")
max_length = cursor.fetchall()
print(max_length)

[(30,)]


In [25]:
# TargetType 테이블 만들기 
cursor.execute("""create table TargetType(
                targtype_code int PRIMARY KEY auto_increment,
                targtype_desc varchar(32)
                )
             """)

In [26]:
# weaptype1 의 max length 체크 
cursor.execute("SELECT max(LENGTH(weaptype1_txt)) FROM origin_terror_data")
max_length = cursor.fetchall()
print(max_length)

[(75,)]


In [27]:
# WeaponType 테이블 만들기 
cursor.execute("""create table WeaponType(
                weaptype_code int PRIMARY KEY auto_increment,
                weaptype_desc varchar(100)
                )
             """)

문제 3-2. origin_terror_data 테이블에서 Attack Type, Target Type, Weapon Type 데이터를 추출하여 문제 3-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요. 
- 데이터를 INSERT 할때 순서를 고민하세요.

제출 5. Table 생성 및 조회 결과 : DESC AttackType, DESC TargetType, WeaponType

제출 6. 관련 코드 및 실행 결과 (ipynb) 

In [45]:
# attacktype1 데이터 추출하기 
cursor.execute("SELECT distinct(attacktype1), attacktype1_txt FROM origin_terror_data ORDER BY attacktype1")
attacktype_result = cursor.fetchall()
attacktype_result = pd.DataFrame(attacktype_result)
attacktype_result

,0,1
0,1,Assassination
1,2,Armed Assault
2,3,Bombing/Explosion
3,4,Hijacking
4,5,Hostage Taking (Barricade Incident)
5,6,Hostage Taking (Kidnapping)
6,7,Facility/Infrastructure Attack
7,8,Unarmed Assault
8,9,Unknown


In [46]:
# AttackType 테이블에 INSERT 
sql = "insert into AttackType(attacktype_code, attacktype_desc) values (%s, %s)"
for i, row in attacktype_result.iterrows():
    cursor.execute(sql, tuple(row))
    print(row)
conn.commit()

0                1
1    Assassination
Name: 0, dtype: object
0                2
1    Armed Assault
Name: 1, dtype: object
0                    3
1    Bombing/Explosion
Name: 2, dtype: object
0            4
1    Hijacking
Name: 3, dtype: object
0                                      5
1    Hostage Taking (Barricade Incident)
Name: 4, dtype: object
0                              6
1    Hostage Taking (Kidnapping)
Name: 5, dtype: object
0                                 7
1    Facility/Infrastructure Attack
Name: 6, dtype: object
0                  8
1    Unarmed Assault
Name: 7, dtype: object
0          9
1    Unknown
Name: 8, dtype: object


In [47]:
# targtype1 데이터 추출하기 
cursor.execute("SELECT distinct(targtype1), targtype1_txt FROM origin_terror_data ORDER BY targtype1")
targtype_result = cursor.fetchall()
targtype_result = pd.DataFrame(targtype_result)
targtype_result

,0,1
0,1,Business
1,2,Government (General)
2,3,Police
3,4,Military
4,5,Abortion Related
5,6,Airports & Aircraft
6,7,Government (Diplomatic)
7,8,Educational Institution
8,9,Food or Water Supply
9,10,Journalists & Media


In [48]:
# TargetType 테이블에 INSERT 
sql = "insert into TargetType(targtype_code, targtype_desc) values (%s, %s)"
for i, row in targtype_result.iterrows():
    cursor.execute(sql, tuple(row))
    print(row)
conn.commit()
# ALTER TABLE tablename
# CHANGE COLUMN old_columnname new_columnname new_datatype;

0           1
1    Business
Name: 0, dtype: object
0                       2
1    Government (General)
Name: 1, dtype: object
0         3
1    Police
Name: 2, dtype: object
0           4
1    Military
Name: 3, dtype: object
0                   5
1    Abortion Related
Name: 4, dtype: object
0                      6
1    Airports & Aircraft
Name: 5, dtype: object
0                          7
1    Government (Diplomatic)
Name: 6, dtype: object
0                          8
1    Educational Institution
Name: 7, dtype: object
0                       9
1    Food or Water Supply
Name: 8, dtype: object
0                     10
1    Journalists & Media
Name: 9, dtype: object
0          11
1    Maritime
Name: 10, dtype: object
0     12
1    NGO
Name: 11, dtype: object
0       13
1    Other
Name: 12, dtype: object
0                             14
1    Private Citizens & Property
Name: 13, dtype: object
0                                15
1    Religious Figures/Institutions
Name: 14, dtype: object


In [49]:
# weaptype1 데이터 추출하기 
cursor.execute("SELECT distinct(weaptype1), weaptype1_txt FROM origin_terror_data ORDER BY weaptype1")
weapontype_result = cursor.fetchall()
weapontype_result = pd.DataFrame(weapontype_result)
weapontype_result

,0,1
0,1,Biological
1,2,Chemical
2,3,Radiological
3,5,Firearms
4,6,Explosives
5,7,Fake Weapons
6,8,Incendiary
7,9,Melee
8,10,Vehicle (not to include vehicle-borne explosiv...
9,11,Sabotage Equipment


In [50]:
# WeaponType 테이블에 INSERT 
sql = "insert into WeaponType(weaptype_code, weaptype_desc) values (%s, %s)"
for i, row in weapontype_result.iterrows():
    cursor.execute(sql, tuple(row))
    print(row)
conn.commit()

0             1
1    Biological
Name: 0, dtype: object
0           2
1    Chemical
Name: 1, dtype: object
0               3
1    Radiological
Name: 2, dtype: object
0           5
1    Firearms
Name: 3, dtype: object
0             6
1    Explosives
Name: 4, dtype: object
0               7
1    Fake Weapons
Name: 5, dtype: object
0             8
1    Incendiary
Name: 6, dtype: object
0        9
1    Melee
Name: 7, dtype: object
0                                                   10
1    Vehicle (not to include vehicle-borne explosiv...
Name: 8, dtype: object
0                    11
1    Sabotage Equipment
Name: 9, dtype: object
0       12
1    Other
Name: 10, dtype: object
0         13
1    Unknown
Name: 11, dtype: object


문제 4. TerrorData 테이블을 만들고 앞서 만들어둔 테이블과 관계를 설정하도록 하겠습니다. 
문제 4-1. TerrorData 테이블을 앞의 그림과 같이 생성하세요. 

- origin_terror_data 를 분석하여 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 

참고> 
- TerrorData.terror id : Auto Increment 
- TerrorData.city_name    = origin_terror_data.city
- TerrorData.target       = orgin_terror_data.target1
- TerrorData.group_name   = origin_terror_data.gname
- TerrorData.kill_count   = origin_terror_data.nkill
- TerrorData.wound_count  = origin_terror_data.nwound
- TerrorData.motive       = origin_terror_data.motive
- TerrorData.summary      = origin_terror_data.summary
- TerrorData.latitude     = origin_terror_data_latitude
- TerrorData.longitude    = origin_terror_data_longitude
- TerrorData.terror_date  = origin_terror_data.iyear + origin_terror_Data.imonth + origin_terror_data.iday (Date Type)

In [51]:
# max length 체크 

# city 의 max length 체크 
cursor.execute("SELECT max(LENGTH(city)) FROM origin_terror_data")
max_length = cursor.fetchall()
print('city : ', max_length)

# target1 의 max length 체크 
cursor.execute("SELECT max(LENGTH(target1)) FROM origin_terror_data")
max_length = cursor.fetchall()
print('target1 : ', max_length)

# gname 의 max length 체크 
cursor.execute("SELECT max(LENGTH(gname)) FROM origin_terror_data")
max_length = cursor.fetchall()
print('gname : ', max_length)

# summary 의 max length 체크 
cursor.execute("SELECT max(LENGTH(summary)) FROM origin_terror_data")
max_length = cursor.fetchall()
print('summary : ', max_length)

# motive 의 max length 체크 
cursor.execute("SELECT max(LENGTH(motive)) FROM origin_terror_data")
max_length = cursor.fetchall()
print('motive : ', max_length) 

# kill_count 의 max length 체크 
cursor.execute("SELECT max(LENGTH(nkill)) FROM origin_terror_data")
max_length = cursor.fetchall()
print('nkill : ', max_length)

# wound_count 의 max length 체크 
cursor.execute("SELECT max(LENGTH(nwound)) FROM origin_terror_data")
max_length = cursor.fetchall()
print('nwound : ', max_length)

# latitude 의 max length 체크 
cursor.execute("SELECT max(LENGTH(latitude)) FROM origin_terror_data")
max_length = cursor.fetchall()
print('latitude : ', max_length)

# longitude 의 max length 체크 
cursor.execute("SELECT max(LENGTH(longitude)) FROM origin_terror_data")
max_length = cursor.fetchall()
print('longitude : ', max_length)

# # terror_date 의 max length 체크 
# cursor.execute("SELECT max(LENGTH(city)) FROM origin_terror_data")
# max_length = cursor.fetchall()
# print('city : ', max_length)

city :  [(65,)]
target1 :  [(343,)]
gname :  [(112,)]
summary :  [(2431,)]
motive :  [(899,)]
nkill :  [(4,)]
nwound :  [(4,)]
latitude :  [(10,)]
longitude :  [(11,)]


In [53]:
# TerrorData 테이블 만들기 
cursor.execute("""create table TerrorData(
                terror_id int PRIMARY KEY auto_increment,
                terror_date date,
                region_code int, foreign key(region_code) references Region(region_code),
                country_code int, foreign key(country_code) references Country(country_code),
                targtype_code int, foreign key(targtype_code) references TargetType(targtype_code),
                attacktype_code int, foreign key(attacktype_code) references AttackType(attacktype_code),
                weaptype_code int, foreign key(weaptype_code) references WeaponType(weaptype_code),
                city varchar(65),
                target varchar(350),
                group_name varchar(120),
                kill_count int,
                wound_count int,
                motive varchar(900),
                summary varchar(2450),
                latitude decimal(16, 14),
                longitude decimal(17, 14)
                )
             """)

문제 4-2. origin_terror_data 테이블에서 Terror Data를 추출하여 문제 4-1.에서 생성한 테이블에 입력하고 확인하세요.

- 앞서 생성한 Region, Country, AttackType, TargetType, WeaponType 데이터와의 관계에 주의하세요.
- Count 값을 가지는 칼럼의 값이 null 인 경우, 0으로 예외처리 해주세요.
- 위도 경도 데이터 중 범위를 넘어서는 데이터가 존재합니다. 이 경우, null 값으로 예외처리 해주세요. (위도 경도 범위 : 구글링해보세요.)
- terror_date 칼럼의 경우, origin_terror_data 의 연, 월, 일 정보를 조합하여 date type 으로 정의해주세요. (Format : ‘YYYY-mm-dd’)
- origin_terror_data 의 월, 일 정보중 값이 0 인 경우 date type 으로 변환되지 않습니다. 이 경우, 1 로 예외처리 해주세요.
- 데이터 입력까지 완료한 이후, origin_terror_data 테이블을 삭제하고 확인하세요.

제출 7. Table 생성 및 조회 결과 : DESC TerrorData, SELECT COUNT(*) FROM TerrorData 

제출 8. 관련 코드 및 실행 결과 (ipynb)

In [85]:
# data 전처리 
# date type : year + month + day (month = 0 인경우 1, day = 0 인경우 1)
# nkill, nwound : null 인 경우 0
# longitude range : 180 ~ -180
# check : select longitude from origin_terror_data where longitude < -180 or longitude > 180; '-86185896'

# terror 데이터 추출하기 
import numpy as np
cursor.execute("SELECT * FROM origin_terror_data")
df_terror = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]
result = [columns, *df_terror]
df_terror = pd.DataFrame(result)
df_terror = df_terror.rename(columns=df_terror.iloc[0]).drop(df_terror.index[0])
df_terror[['nkill', 'nwound']].fillna(0)
df_terror['latitude'] = df_terror['latitude'].apply(lambda x: np.NaN if (x is not None) and (x > 90 or x < -90) else x)
df_terror['longitude'] = df_terror['longitude'].apply(lambda x: np.NaN if (x is not None) and (x > 180 or x < -180) else x)

df_terror['imonth'] = df_terror['imonth'].apply(lambda x: 1 if (x is not None) and (x == 0) else x)
df_terror['iday'] = df_terror['iday'].apply(lambda x: 1 if (x is not None) and (x == 0) else x)

df_terror['terror_date'] = pd.to_datetime(df_terror[['iyear', 'imonth', 'iday']].astype(str).apply('-'.join, axis=1))
df_terror[['terror_date', 'region', 'country', 'targtype1', 'attacktype1', 'weaptype1', 'city', 'target1', 'gname', 'nkill', 'nwound', 'motive', 'summary', 'latitude', 'longitude']]

,terror_date,region,country,targtype1,attacktype1,weaptype1,city,target1,gname,nkill,nwound,motive,summary,latitude,longitude
1,1970-07-02,2,58,14,1,13,Santo Domingo,Julio Guzman,MANO-D,1.0,0.0,None,None,18.456792,-69.951164
2,1970-01-01,1,130,7,6,13,Mexico city,"Nadine Chaval, daughter",23rd of September Communist League,0.0,0.0,None,None,19.371887,-99.086624
3,1970-01-01,5,160,10,1,13,Unknown,Employee,Unknown,1.0,0.0,None,None,15.478598,120.599741
4,1970-01-01,8,78,7,3,6,Athens,U.S. Embassy,Unknown,None,None,None,None,37.997490,23.762728
5,1970-01-01,4,101,7,7,8,Fukouka,U.S. Consulate,Unknown,None,None,None,None,33.580412,130.396361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181687,2017-12-31,11,182,4,2,5,Ceelka Geelow,Checkpoint,Al-Shabaab,1.0,2.0,None,12/31/2017: Assailants opened fire on a Somali...,2.359673,45.385034
181688,2017-12-31,10,200,4,3,6,Jableh,Hmeymim Air Base,Muslim extremists,2.0,7.0,None,12/31/2017: Assailants launched mortars at the...,35.407278,35.942679
181689,2017-12-31,5,160,14,7,8,Kubentog,Houses,Bangsamoro Islamic Freedom Movement (BIFM),0.0,0.0,None,12/31/2017: Assailants set fire to houses in K...,6.900742,124.437908
181690,2017-12-31,6,92,2,3,6,Imphal,Office,Unknown,0.0,0.0,None,12/31/2017: Assailants threw a grenade at a Fo...,24.798346,93.940430


In [111]:
cursor.execute("DESC TerrorData")
rows = cursor.fetchall()
for row in rows:
    print(row)
cursor.execute("SELECT COUNT(*) FROM TerrorData")
rows = cursor.fetchall()
for row in rows:
    print(row)

('terror_id', b'int', 'NO', 'PRI', None, 'auto_increment')
('terror_date', b'date', 'YES', '', None, '')
('region_code', b'int', 'YES', 'MUL', None, '')
('country_code', b'int', 'YES', 'MUL', None, '')
('targtype_code', b'int', 'YES', 'MUL', None, '')
('attacktype_code', b'int', 'YES', 'MUL', None, '')
('weaptype_code', b'int', 'YES', 'MUL', None, '')
('city', b'varchar(65)', 'YES', '', None, '')
('target', b'varchar(350)', 'YES', '', None, '')
('group_name', b'varchar(120)', 'YES', '', None, '')
('kill_count', b'int', 'YES', '', None, '')
('wound_count', b'int', 'YES', '', None, '')
('motive', b'varchar(900)', 'YES', '', None, '')
('summary', b'varchar(2450)', 'YES', '', None, '')
('latitude', b'decimal(16,14)', 'YES', '', None, '')
('longitude', b'decimal(17,14)', 'YES', '', None, '')
(181707,)


In [91]:
# TerrorData 테이블에 INSERT 
df_terror = df_terror[['terror_date', 'region', 'country', 'targtype1', 'attacktype1', 'weaptype1', 'city', 'target1', 'gname', 'nkill', 'nwound', 'motive', 'summary', 'latitude', 'longitude']]
df_terror.fillna
sql = "insert into TerrorData(terror_date, region_code, country_code, targtype_code, attacktype_code, weaptype_code, city, target, group_name, kill_count, wound_count, motive, summary, latitude, longitude) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

def convert_nan_to_none(value):
    return None if pd.isna(value) else value

for i, row in df_terror.iterrows():
    row = list(row.apply(convert_nan_to_none))
    cursor.execute(sql, tuple(row))
conn.commit()

TypeError: 'NoneType' object is not subscriptable

In [114]:
# 제출용 csv로 저장
df_terror.to_csv('sql3_TerrorData.csv', sep=',', header=False, index=False, encoding='utf8')

문제 5. TerrorData 의 전체 기간에서 테러의 숫자를 연도별로 집계하여 연도별 테러 발생 건수를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 

제출 9. 관련 코드 및 실행 결과 (ipynb)

In [94]:
cursor.execute("SELECT YEAR(terror_date), count(terror_id) FROM TerrorData GROUP BY YEAR(terror_date)")
terror_per_year = cursor.fetchall()
print(terror_per_year)

[(1970, 667), (1971, 471), (1972, 568), (1973, 473), (1974, 581), (1975, 740), (1976, 923), (1977, 1319), (1978, 1526), (1979, 2662), (1980, 2662), (1981, 2586), (1986, 2860), (1982, 2544), (1983, 2870), (1984, 3495), (1985, 2915), (1987, 3183), (1988, 3721), (1989, 4324), (1990, 3887), (1991, 4683), (1992, 5071), (1994, 3456), (1995, 3081), (1996, 3058), (1997, 3197), (1998, 934), (1999, 1395), (2000, 1814), (2001, 1906), (2002, 1333), (2003, 1278), (2004, 1166), (2005, 2017), (2006, 2758), (2007, 3242), (2008, 4805), (2009, 4721), (2010, 4826), (2011, 5076), (2012, 8522), (2013, 12036), (2014, 16903), (2015, 14965), (2016, 13587), (2017, 10900)]


문제 6. TerrorData 에서 테러가 가장 많이 일어난 순서로 국가를 정렬하여 상위 10위 국가를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 국가 정보는 이름으로 출력하세요.

제출 10. 관련 코드 및 실행 결과 (ipynb)

In [107]:
# 테러가 많이 일어난 상위 10위 Region
cursor.execute("""select r.region_name, count(a.terror_id)
                  from TerrorData a
                  join Region r
                  on a.region_code = r.region_code
                  group by a.region_code
                  order by count(a.terror_id) desc
                  limit 10;
                """)
terror_region_top_10 = cursor.fetchall()
conn.commit()
print('테러가 많이 일어나는 상위 10개 Region')
for row in terror_region_top_10:
    print(row)
    
print('')

# 테러가 많이 일어난 상위 10위 Country 
cursor.execute("""select c.country_name, count(a.terror_id)
                  from TerrorData a
                  join Country c
                  on a.country_code = c.country_code
                  group by a.country_code
                  order by count(a.terror_id) desc
                  limit 10;
                """)
terror_country_top_10 = cursor.fetchall()
conn.commit()
print('테러가 많이 일어나는 상위 10개 Country')
for row in terror_country_top_10:
    print(row)

테러가 많이 일어나는 상위 10개 Region
('Middle East & North Africa', 50474)
('South Asia', 44974)
('South America', 18979)
('Sub-Saharan Africa', 17550)
('Western Europe', 16641)
('Southeast Asia', 12486)
('Central America & Caribbean', 10345)
('Eastern Europe', 5145)
('North America', 3465)
('East Asia', 803)

테러가 많이 일어나는 상위 10개 Country
('Iraq', 24636)
('Pakistan', 14368)
('Afghanistan', 12731)
('India', 11960)
('Colombia', 8306)
('Philippines', 6909)
('Peru', 6096)
('El Salvador', 5320)
('United Kingdom', 5235)
('Turkey', 4292)


문제 7. TerrorData 에서 테러가 가장 많이 일어난 상위 10위 국가에 대해 국가별로 사망자수와 부상자수, 사상자수(사망자수 + 부상자수)를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 국가 정보는 이름으로 출력하세요.

제출 11. 관련 코드 및 실행 결과 (ipynb)

In [108]:
cursor.execute("""select c.country_name, count(a.terror_id), sum(a.kill_count), sum(a.wound_count), (sum(a.kill_count) + sum(a.wound_count)) as casualties
                  from TerrorData a
                  join Country c
                  on a.country_code = c.country_code
                  group by a.country_code
                  order by count(a.terror_id) desc
                  limit 10;
                """)
terror_top_10_casualties = cursor.fetchall()
conn.commit()
for row in terror_top_10_casualties:
    print(row)

('Iraq', 24636, Decimal('78589'), Decimal('134690'), Decimal('213279'))
('Pakistan', 14368, Decimal('23822'), Decimal('42038'), Decimal('65860'))
('Afghanistan', 12731, Decimal('39384'), Decimal('44277'), Decimal('83661'))
('India', 11960, Decimal('19341'), Decimal('28980'), Decimal('48321'))
('Colombia', 8306, Decimal('14698'), Decimal('10328'), Decimal('25026'))
('Philippines', 6909, Decimal('9560'), Decimal('13367'), Decimal('22927'))
('Peru', 6096, Decimal('12771'), Decimal('4078'), Decimal('16849'))
('El Salvador', 5320, Decimal('12053'), Decimal('5062'), Decimal('17115'))
('United Kingdom', 5235, Decimal('3410'), Decimal('6106'), Decimal('9516'))
('Turkey', 4292, Decimal('6888'), Decimal('9899'), Decimal('16787'))


문제 8. 지역별 테러 공격 형태에 따른 사망자수, 부상자수, 사상자수를 조회하세요.

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 지역 정보(Region)는 이름으로 출력하세요.

제출 12. 관련 코드 및 실행 결과 (ipynb)

In [109]:
# 지역별 테러 공격 형태에 따른 사망자와 사상자 수 
cursor.execute("""select r.region_name, `at`.attacktype_desc, sum(a.kill_count), sum(a.wound_count), (sum(a.kill_count) + sum(a.wound_count)) as casualties
                  from TerrorData a
                  join Region r
                  on a.region_code = r.region_code
                  join AttackType `at`
                  on a.attacktype_code = `at`.attacktype_code
                  group by a.region_code, a.attacktype_code
                """)
region_attacktype_casualties = cursor.fetchall()
conn.commit()
for row in region_attacktype_casualties:
    print(row)

('North America', 'Assassination', Decimal('238'), Decimal('84'), Decimal('322'))
('North America', 'Armed Assault', Decimal('837'), Decimal('1542'), Decimal('2379'))
('North America', 'Bombing/Explosion', Decimal('598'), Decimal('1806'), Decimal('2404'))
('North America', 'Hijacking', Decimal('3005'), Decimal('16512'), Decimal('19517'))
('North America', 'Hostage Taking (Barricade Incident)', Decimal('81'), Decimal('80'), Decimal('161'))
('North America', 'Hostage Taking (Kidnapping)', Decimal('67'), Decimal('52'), Decimal('119'))
('North America', 'Facility/Infrastructure Attack', Decimal('64'), Decimal('460'), Decimal('524'))
('North America', 'Unarmed Assault', Decimal('16'), Decimal('970'), Decimal('986'))
('North America', 'Unknown', Decimal('10'), Decimal('25'), Decimal('35'))
('Central America & Caribbean', 'Assassination', Decimal('1700'), Decimal('246'), Decimal('1946'))
('Central America & Caribbean', 'Armed Assault', Decimal('23233'), Decimal('5459'), Decimal('28692'))
('Ce

---

# reference

민형동 (2020). Global Terrorism Data를 활용한 국가별 테러유형 분석. 한국공안행정학회, 82, 63-84. 

한인택 (2017). Global Terrorism Database를 활용한 테러리즘의 현황분석. 

민형기 (2019). 전세계 테러 데이터 분석 (따라하기). https://pinkwink.kr/1214 

ASHWINI SWSAIN (2018). Terrorism Around The World. https://www.kaggle.com/code/ash316/terrorism-around-the-world/notebook

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021). *Global Terrorism Database* (GTD) [Data set]. University of Maryland. https://www.start.umd.edu/gtd

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021, August). *Global Terrorism Database codebook: Methodology, inclusion criteria, and variables.* University of Maryland. https://www.start.umd.edu/gtd/downloads/Codebook.pdf 
